In [0]:
%scala
import org.apache.spark.sql.SparkSession
import org.apache.spark.sql.functions._
import org.apache.spark.graphx.{Graph, Edge}
import org.apache.spark.rdd.RDD
import scala.reflect.ClassTag
import org.apache.spark.graphx._
import scala.reflect.ClassTag
import org.apache.spark.graphx._
import org.apache.spark.sql.functions._
import spark.implicits._




In [0]:
%scala
// Create a SparkSession
val spark = SparkSession.builder().appName("Graphs").getOrCreate()


In [0]:
%scala
val londonArea = spark.sql("""
  SELECT b.name, ST_Buffer(b.geometry, 5) AS geometry
  FROM geospatial.lookups.boundary_line_ceremonial_counties b 
  WHERE b.name IN ('City and County of the City of London')
""")

londonArea.cache()
londonArea.createOrReplaceTempView("london_area_vw")
londonArea.count()

In [0]:
%scala
val londonRoadNodes = spark.sql("""
SELECT a.fid, a.id, a.form_of_road_node, a.geometry
FROM geospatial.networks.road_node a
INNER JOIN london_area_vw b 
ON st_within(a.geometry, b.geometry)
""")
londonRoadNodes.cache()
londonRoadNodes.createOrReplaceTempView("london_road_nodes_vw")
londonRoadNodes.count()

In [0]:
%scala
londonRoadNodes.count()

In [0]:
%scala
londonRoadNodes.printSchema

In [0]:
%scala
val londonRoadEdges = spark.sql("""
SELECT a.fid, a.id, b.fid AS src, c.fid AS dst, a.length AS distance
FROM geospatial.networks.road_link a
LEFT JOIN london_road_nodes_vw b
ON a.start_node = b.id
LEFT JOIN london_road_nodes_vw c
ON a.end_node = c.id
WHERE b.id IS NOT NULL
AND c.id IS NOT NULL
UNION ALL
SELECT a.fid, a.id, c.fid AS src, b.fid AS dst, a.length AS distance
FROM geospatial.networks.road_link a
LEFT JOIN london_road_nodes_vw b
ON a.start_node = b.id
LEFT JOIN london_road_nodes_vw c
ON a.end_node = c.id
WHERE b.id IS NOT NULL
AND c.id IS NOT NULL
""")
londonRoadEdges.cache()
londonRoadEdges.createOrReplaceTempView("london_road_edges_vw")
londonRoadEdges.count()

In [0]:
%scala
londonRoadEdges.printSchema

In [0]:
%scala
londonRoadEdges.count()

In [0]:
%scala

// Define vertices: (id, name)
val vertexDF = londonRoadNodes.select(
  col("fid").cast("long").alias("id"),
  col("id").alias("name")
)

// Define edges: (src, dst, distance)
val edgeDF = londonRoadEdges.select(
  col("src").cast("long").alias("src"),
  col("dst").cast("long").alias("dst"),
  col("distance")
)

val vertexRDD: RDD[(Long, String)] = vertexDF.rdd.map(row => (row.getAs[Long]("id"), row.getAs[String]("name")))
val edgeRDD: RDD[Edge[Double]] = edgeDF.rdd.map(row => 
  Edge(row.getAs[Long]("src"), row.getAs[Long]("dst"), row.getAs[Double]("distance"))
)

val graph: Graph[String, Double] = Graph(vertexRDD, edgeRDD)

In [0]:
%scala
val edgesDF = graph.edges.toDF
val verticesDF = graph.vertices.toDF

In [0]:
%scala
val topCandidateIds = Seq(1902943).map(_.toLong)


In [0]:
%scala
def multiSourceDijkstra[VD](
    g: Graph[VD, Double], 
    origins: Seq[VertexId]
): Graph[(Double, VertexId, List[VertexId]), Double] = {  // Now includes origin ID
  // Initialize: (distance, origin, path)
  val initialGraph = g.mapVertices((id, _) =>
    if (origins.contains(id)) (0.0, id, List(id))  // origin vertex: distance=0, origin=itself, path=[id]
    else (Double.PositiveInfinity, -1L, List())    // others: infinity, no origin, empty path
  )

  val sssp = initialGraph.pregel((Double.PositiveInfinity, -1L, List[VertexId]()))(
    // Vertex program: keep the path with smallest distance
    (id, current, newUpdate) => 
      if (current._1 < newUpdate._1) current else newUpdate,

    // Send messages to neighbors if a shorter path is found
    triplet => {
      val newDist = triplet.srcAttr._1 + triplet.attr
      if (newDist < triplet.dstAttr._1) {
        Iterator((triplet.dstId, (newDist, triplet.srcAttr._2, triplet.srcAttr._3 :+ triplet.dstId)))
      } else {
        Iterator.empty
      }
    },

    // Merge messages: keep the one with the smallest distance
    (a, b) => if (a._1 < b._1) a else b
  )

  sssp
}

// Run Dijkstra's algorithm
val result = multiSourceDijkstra(graph.mapEdges(e => e.attr), topCandidateIds)

In [0]:
%scala
val resultDF = result.vertices.map { case (id, (distance, origin, path)) =>
  (origin, id, distance, path.mkString(", "))  // (start, end, distance, path)
}.toDF("start_point", "end_point", "distance", "path")

// Order by distance in descending order (or ascending, if preferred)
val orderedResultDF = resultDF.orderBy($"distance".desc)

// Display the result DataFrame
display(orderedResultDF)

In [0]:
%scala
val catalogName = "geospatial"
val schema = "shortest_path"
val catalogContainerName = "geospatial-dataset"
val catalogStorageAccountName = "melikadatabricksstorage"

val sqlQuery = s"""
  CREATE SCHEMA IF NOT EXISTS ${catalogName}.${schema}
  COMMENT 'This schema contains ${schema} data of the UK'
  MANAGED LOCATION 'abfss://${catalogContainerName}@${catalogStorageAccountName}.dfs.core.windows.net/'
"""

spark.sql(sqlQuery)

In [0]:
%scala


topCandidateIds.foreach { startId =>
  val pathsForStartId = orderedResultDF.filter($"start_point" === startId)
  
  val tableName = s"start_point_$startId"
  
  pathsForStartId.write
  .format("delta")
  .mode("overwrite") // Choose the appropriate write mode
  .option("overwriteSchema", "true") // Optional: overwrite schema if it changes
  .saveAsTable(s"${catalogName}.${schema}.${tableName}")
}